In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [5]:
import sys
sys.path.append("/home/tom/coding/ExTRApack/")
import ExTRA as ex

ExTRA correctly imported


In [6]:
#standard epochs
J1991=2448349.0 #1991.25
J2000=2451545.0
J2016=2457389.0

In [7]:
#standard model:
#at J2016

asc=20
dec=-10
parallax=200
mu_a_star=-100
mu_d=+300

standard=np.array([asc,dec,parallax,mu_a_star,mu_d])

#hip_inaccuracy=np.array([1,1,0,-5,+5])
hip_inaccuracy=np.zeros(5)

In [8]:
#orbital elements
P=550
e=0.3
om=np.radians(0)
i=np.radians(30)
Om=np.radians(75)
T0=J1991+900 #900 days after standard 1991
a=3
orbpar=np.array([P,e,om,i,Om,T0,a])


#timestamps of measurements
t_mock=np.linspace(T0-2*P,T0+2*P,100)
#orbit
t_orb=np.linspace(T0-2*P,T0+2*P,int(P/5))
mock_orbit=ex.orbit(*orbpar,t_orb)

# RV data generation

In [9]:
#rv
t_RV=np.linspace(T0-2*P,T0+2*P,int(P/5))

v=10

#RVdata
RV=ex.RV_comb(v,P,e,om,i,T0,a,parallax,t_RV)
#error
RV_err=np.ones(len(RV))*10



#plt.figure(figsize=(10,5))
#plt.scatter(t_RV,RV,s=10)
#plt.xlim([t_RV[0],t_RV[-1]])
#plt.grid()
#plt.show()


In [10]:
def write_RV(data):
    with open('mock_RV.txt', mode='w') as file:
        for row in data:
            file.write("{:<10}\t{:<10}\t{:<10}\n".format(row[0], row[1], row[2]))

In [11]:
RV_data=np.array([t_RV,RV,RV_err])
RV_data=np.transpose(RV_data)
write_RV(RV_data)

# Hip data generation

In [1]:
#HIP
#motions
t_HIP=t_RV
#t_HIP=np.linspace(T0,T0+600,30)

earth=ex.earth_position(t_HIP)

std=np.array(ex.standard_model(asc,dec,parallax,mu_a_star,mu_d,t_HIP,earth,tangential=True))
orb=np.array(ex.orbit(P,e,om,i,Om,T0,a,t_HIP))
#motion=std+orb




#A3,A4,A5,A6,A7,A8,A9=hip_ad

#Angle of measurement

angle=np.arctan2(orb[1],orb[0]) # use this to have an easy example


#angle=np.random.uniform(0,2*np.pi,len(t_HIP)) #use this for a random angle distribution



############################################################################################ dont touch after this line
A8=np.cos(angle)*orb[0]+np.sin(angle)*orb[1]
A9=abs(np.random.normal(np.std(A8),0.5*np.std(A8),len(A8)))

#A9=np.ones(len(A8))*np.std(A8)*0.25

A5=np.zeros(len(A8))


#X and Y
A3=1*np.cos(angle)
A4=1*np.sin(angle)

#year of mesaurement
epoch=(t_HIP-J2000)/365.25 +2000

#years after HIP stand epoch
frac=epoch-1991.25
A6=frac*A3
A7=frac*A4 #to A4 corresponding poper motion factor


HIP=A3,A4,A5,A6,A7,A8,A9

hip_standard=np.zeros(5)


hip_standard[0],hip_standard[1]=ex.pos_recalc(standard,J2016,J1991)
#hip standard epoch is J1991


hip_standard[2:5]=np.array(standard[2:5])
##################################################################################### end of data generation



NameError: name 't_RV' is not defined

In [13]:
#x,xe,y,ye=ex.hip_2d(A3,A4,A8,A9)
#plt.scatter(0,0)
#ex.plot_hip(x,xe,y,ye)
#plt.scatter(orb[0],orb[1],c="k")
#plt.axis("equal")
#plt.grid()
#plt.show()

In [14]:
#writing the data
data=np.array([np.arange(0,len(A8)),frac,A5,A3,A4,A8,A9])
data=np.around(data,5)
data=np.transpose(data)

In [15]:
def write_HIP(data):
    with open('mock_HIP.txt', mode='w') as file:
        file.write("useless one line header \n")
        for row in data:
            file.write("{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\n".format(
                row[0], row[1], row[2],row[3],row[4],row[5],row[6]))


In [16]:
write_HIP(data)

In [17]:
par= np.array([format(x, '.10g') for x in orbpar])
variables=standard,par

In [18]:
def write_solution(variables):
    with open('sol.txt', mode='w') as file:
        file.write('Gaia parameters:\n')
        file.write(str(variables[0])+"\n")
        file.write('Orbit parameters:\n')
        file.write(str(variables[1]))


In [19]:
write_solution(variables)

In [20]:
def planet_mass(M_s,parallax,P,e,i,a):
    
    K1=2*np.pi*a*np.sin(i)*1.496e11/(parallax)
    K2=(P*86400*(1-e**2)**0.5)
    K=K1/K2
    G=6.6743*1e-11
    print("K:")
    print(K)
    #print((2*np.pi*G)*(M_s**2))
    M_p=((K**3) *(P*24*60**2)/(2*np.pi*G*np.sin(i)**3)*(M_s**2))**(1/3)
    return M_p/M_Jup

In [21]:
def K(parallax,P,e,i,a):
    K1=2*np.pi*a*np.sin(i)*1.496e11/(parallax)
    K2=(P*86400*(1-e**2)**0.5)
    K=K1/K2

    return K


In [28]:
K(parallax,P,e,i,a)

155.51618614599337

In [26]:
parallax

200

In [ ]:
def a_mas(parallax,K,P,e,i):
    